# Streaming Multiple Motion Capture Markers
In this notebook, multiple motion capture markers are streamed, where each marker represents a channel whose data is structured string

In [ ]:
import asyncio
import qtm
from pylsl import StreamInfo, StreamOutlet, local_clock

In [2]:
n_channels = 6
info = StreamInfo('QTM_multi', 'Motion Capture', n_channels, 300, 3, 'multi5')
outlet = StreamOutlet(info)

In [3]:
def on_packet(packet):
    print("\nFramenumber: {}".format(packet.framenumber))
    header, markers = packet.get_3d_markers()
    counter = 1
    dictionary = {}
    for marker in markers:
        #print (marker)
        coor = str(marker)[19:-1]
        key = 'marker' + str(counter)
        dictionary[key] = coor
        counter += 1
        
    print(dictionary)
    print (local_clock(), '\n')
    #print(list(dictionary.values()))
    push = list(dictionary.values())
    push.append(format(packet.framenumber))
    outlet.push_chunk(push)
#     outlet.push_chunk(list(dictionary.values()))

In [4]:
async def setup():
    """ Main function """
    connection = await qtm.connect("127.0.0.1")
    if connection is None:
        return

    await connection.stream_frames(components=["3d"], on_packet=on_packet)

In [ ]:
asyncio.ensure_future(setup())
asyncio.get_event_loop().run_forever()